In [ ]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades
from elfpy.utils.plot import format_axis, annotate

In [ ]:
# make folder if it doesn't exit
out_dir = os.path.join(os.pardir, 'outputs')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
pd.set_option("float_format",'{:,.6f}'.format)

random_seed = 3
simulator_rng = np.random.default_rng(random_seed)
config = {
    "min_fee": 0.1, # decimal that assigns fee_percent
    "max_fee": 0.5, # decimal that assigns fee_percent
    "min_target_liquidity": 1e6, # in USD
    "max_target_liquidity": 10e6, # in USD
    "min_target_volume": 0.001, # fraction of pool liquidity
    "max_target_volume": 0.01, # fration of pool liquidity
    "min_pool_apy": 0.02, # as a decimal
    "max_pool_apy": 0.9, # as a decimal
    "pool_apy_target_range": [0.15,0.20], # as a decimal
    "pool_apy_target_range_convergence_speed": 0.52, # as a share of trades that move in convergence direction
    "min_vault_age": 0, # fraction of a year
    "max_vault_age": 1, # fraction of a year
    "min_vault_apy": 0.001, # as a decimal
    "max_vault_apy": 0.9, # as a decimal
    "base_asset_price": 2.5e3, # aka market price
    "pool_duration": 180, # in days
    "num_trading_days": 180, # should be <= pool_duration
    "floor_fee": 0, # minimum fee percentage (bps)
    "tokens": ["base", "fyt"],
    "trade_direction": "out",
    "precision": None,
    "rng": simulator_rng,
    "verbose": False,
    'pricing_model_name': 'YieldSpacev2',
}

In [ ]:
start_time = time.time()
num_runs = 0

simulator = YieldSimulator(**config)
simulator.set_random_variables()

scenarios = {
    "start_apy": [0.1,0.1],  # in decimal
    'pool_apy_target_range': [[0.15,0.20],[0.0,0.05]], # as a decimal
    "pool_apy_target_range_convergence_speed": [0.52,0.52], # as a share of trades that move in convergence direction
    'name': ['spike','drop'],
}
for idx in range(0, len(scenarios["start_apy"])):
    override_dict = {
        'init_vault_age': 1,
        'vault_apy': [scenarios["start_apy"][idx]]*config['num_trading_days'],
        'init_pool_apy': scenarios["start_apy"][idx],
        'pool_apy_target_range': scenarios["pool_apy_target_range"][idx],
        'pool_apy_target_range_convergence_speed': scenarios["pool_apy_target_range_convergence_speed"][idx],
        'target_daily_volume': 5*1e5,
        'target_liquidity': 10*1e6,
        'scenario_name': scenarios['name'][idx],
        'fee_percent' : 0.1,
    }
    simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
    simulator.run_simulation(override_dict)
    num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
[trades, trades_agg] = format_trades(simulator.analysis_dict)
# display(trades.head(1).T)

In [ ]:
display(trades.groupby(['scenario_name', 'token_in']).agg({
    'trade_volume_usd': ['sum'],
    'run_trade_number': ['count'],
    'fee_in_bps': ['mean', 'std', 'min', 'max', 'sum'],
    'day': ['mean', 'min', 'max'],
    })
)
# print(trades_agg.columns)

In [ ]:
num_plots = 2
colors = list(plt.rcParams['axes.prop_cycle'].by_key()['color'])
model_colors = {model:colors[i] for i, model in enumerate(trades_agg.scenario_name.unique())}

## DAY PLOTS
figsize = (24, 12)
fig, ax = plt.subplots(
    ncols=1,
    nrows=num_plots,
    gridspec_kw={'wspace':0, 'hspace':0, 'height_ratios':np.ones(num_plots)}
)
fig.patch.set_facecolor('white')   # set fig background color to white

# fees
current_plot = 0
for model in trades_agg.scenario_name.unique():
    model_df = trades_agg.loc[trades_agg.scenario_name==model, :]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=model
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y'
)
ax[current_plot].xaxis.set_ticklabels([])
title = (
    f'Initial pool APY: {trades.pool_apy[0]*100:.2f}% '
    + f'Time Stretch: {trades.init_time_stretch[0]:.2f} '
    + f'Maturity: {trades.pool_duration[0]} days\n'
    + f'Target Liquidity: {trades.target_liquidity[0]:,.0f} '
    + f'Target Daily Volume: {trades.target_daily_volume[0]:,.0f} '
    + f'Percent Fees: {trades.fee_percent[0]:.2f}%'
)
ax[current_plot].set_title(title, fontsize=20)
ax[current_plot].legend(fontsize=18)

# fees cumulative
current_plot += 1
for model_idx, model in enumerate(trades_agg.scenario_name.unique()):
    model_df = trades_agg.loc[trades_agg.scenario_name==model, :]
    #cumulative_fee = model_df.fee_in_usd_cum_sum.iloc[-1]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_cum_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        legend=None,
    )
    xlim = ax[current_plot].get_xlim()
    total_fees = model_df.fee_in_usd_cum_sum.iloc[-1]
    ax[current_plot].annotate(
        f'{model} total fees collected = {total_fees:,.0f} USD',
        (xlim[0] + (xlim[1]-xlim[0])*0.6,
        1.01e5-2.5e4*model_idx),
        fontsize=18
    )
ax[current_plot].set_ylabel('Cumulative Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y',
)
ax[current_plot].set_xlabel('Day', fontsize=18)

fname = 'rate_scenarios_fees.png'
fig.savefig(os.path.join(out_dir, fname),bbox_inches='tight')
print(f'Figure saved to {os.path.join(out_dir, fname)}')

In [ ]:
## TRADE PLOTS
debug_convergence_parameters = 0  # set this to 2 to plot additional convergence parameters
plot_annualized = 1
num_plots = 2 + debug_convergence_parameters + plot_annualized
figsize = (24, 6*num_plots)
fig, ax = plt.subplots(
    ncols=1,
    nrows=num_plots,
    gridspec_kw={'wspace':0, 'hspace':0, 'height_ratios':np.ones(num_plots)}
)
fig.patch.set_facecolor('white')   # set fig background color to white

### pool & vault APYs ###
current_plot = 0
for model_idx, model in enumerate(trades.scenario_name.unique()): # for each model (per run?)
    model_df = trades.loc[trades.scenario_name==model, :]
    ax[current_plot] = model_df.plot(
        x='run_trade_number', # could also do num_orders if you want to know the apy for a given volume of trade
        y='pool_apy_percent',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=f'{model}'
    )
    if model_idx == len(trades.scenario_name.unique()) - 1:
        label = 'vault'
    else:
        label = '_nolegend_'
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='vault_apy_percent',
        figsize=figsize,
        ax=ax[current_plot],
        linestyle='-',
        color='black',
        label=label,
    )
ax[current_plot].set_ylabel('APY (percent)', fontsize=18)
if ax[current_plot].get_ylim()[0] > 0:
    ax[current_plot].set_ylim(0, ax[current_plot].get_ylim()[1]) # set y axis to start at 0 if it's positive
format_axis(ax[current_plot])

### direction parameters ###
if debug_convergence_parameters>0:
    current_plot += 1
    for model_idx, model in enumerate(trades.scenario_name.unique()):
        model_df = trades.loc[trades.scenario_name==model, :]
        if model_idx==0:
            ax[current_plot] = model_df.plot(
                x='run_trade_number',
                y=['apy_distance_in_target_range','apy_distance_from_mid_when_in_range'],
                figsize=figsize,
                ax=ax[current_plot],
                label=['apy_distance_in_target_range','apy_distance_from_mid_when_in_range'],
            )
    if ax[current_plot].get_ylim()[0] > 0:
        ax[current_plot].set_ylim(0, ax[current_plot].get_ylim()[1]) # set y axis to start at 0 if it's positive

    current_plot += 1
    for model_idx, model in enumerate(trades.scenario_name.unique()):
        model_df = trades.loc[trades.scenario_name==model, :]
        if model_idx==0:
            ax[current_plot] = model_df.plot(
                x='run_trade_number',
                y=['actual_convergence_strength'],
                figsize=figsize,
                ax=ax[current_plot],
                label=['actual_convergence_strength'],
            )
            model_df.plot.scatter(
                x='run_trade_number',
                y='streak_luck',
                s='streak_luck',
                figsize=figsize,
                ax=ax[current_plot],
                label='streak_luck',
                alpha=1,
                color='orange',
            )
format_axis(ax[current_plot])

### annualized return ###
if plot_annualized > 0:
    current_plot += 1
    annotate_dict = {'major_offset': 0.06/num_plots*2,'minor_offset': 0.026/num_plots*2,
    'position_y': 0.48+(num_plots-1)*0.12-current_plot*0.25,  # how far up the y axis to put the annotation
    'position_x': 0.77,  # how far down the x axis to put the annotation
    'font_size': 16, 'alpha': 1}
    for model_idx, model in enumerate(trades.scenario_name.unique()):
        model_df = trades.loc[trades.scenario_name==model, :]
        ax[current_plot] = model_df.plot(
            x='run_trade_number',
            y=['share_price_total_return_percent_annualized','price_total_return_percent_annualized','lp_total_return_percent_annualized'],
            figsize=figsize,
            ax=ax[current_plot],
            style=['--','-',':'],
            color=model_colors[model],
            label=['Base/Vault','Principal Token','Fees accrued to LPs'],
        )
        annotate(ax[current_plot], f'{model:5} vault/base peak APR = {max(model_df.loc[:, "share_price_total_return_percent_annualized"].values):,.1f}%', 0, model_idx, annotate_dict)
        annotate(ax[current_plot], f'{model:5} PT peak APR = {max(model_df.loc[:, "price_total_return_percent_annualized"].values):,.1f}%', 1, model_idx, annotate_dict)
        annotate(ax[current_plot], f'{model:5} LP fee peak APR = {max(model_df.loc[:, "lp_total_return_percent_annualized"].values):,.1f}%', 2, model_idx, annotate_dict)
    ax[current_plot].set_ylabel('APR (%)', fontsize=18)
    format_axis(ax[current_plot])
    handles,lines = ax[current_plot].get_legend_handles_labels()
    leg = ax[current_plot].legend([lines[0], lines[1], lines[2]], fontsize=18)
    for l in leg.legendHandles: l.set_color('black')

### price per share & spot price ###
current_plot += 1
annotate_dict = {'major_offset': 0.06/num_plots*2,'minor_offset': 0.026/num_plots*2,
    'position_y': 0.48+(num_plots-1)*0.12-current_plot*0.25,  # how far up the y axis to put the annotation
    'position_x': 0.77,  # how far down the x axis to put the annotation
    'font_size': 16, 'alpha': 1}
for model_idx, model in enumerate(trades.scenario_name.unique()):
    model_df = trades.loc[trades.scenario_name==model, :]
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y=['share_price_total_return_percent','price_total_return_percent','lp_total_return_percent'],
        figsize=figsize,
        ax=ax[current_plot],
        style=['--','-',':'],
        color=model_colors[model],
        label=['Base/Vault','Principal Token','Fees accrued to LPs'],
    )
    annotate(ax[current_plot], f'{model:5} vault/base return = {model_df.loc[:, "share_price_total_return"].values[-1]:.3%}', 0, model_idx, annotate_dict)
    annotate(ax[current_plot], f'{model:5} PT return = {model_df.loc[:, "price_total_return"].values[-1]:.3%}', 1, model_idx, annotate_dict)
    annotate(ax[current_plot], f'{model:5} LP fee return = {model_df.loc[:, "lp_total_return"].values[-1]:.3%}', 2, model_idx, annotate_dict)
ax[current_plot].set_ylabel('Holding Period Return (%)', fontsize=18)
format_axis(ax[current_plot])
handles,lines = ax[current_plot].get_legend_handles_labels()
leg = ax[current_plot].legend([lines[0], lines[1], lines[2]], fontsize=18)
for l in leg.legendHandles: l.set_color('black')

### format x-axis only after the last plot ###
ax[current_plot].set_xlabel('Day', fontsize=18)
xticks = np.append(np.insert(np.arange(30,config['pool_duration'],30),0,0),config['pool_duration']-1)
xtickdata = trades.groupby('day')['run_trade_number'].mean()
xtickdata = xtickdata.loc[[x for x in xtickdata.index if xtickdata.index[x] in xticks]]
ax[current_plot].set_xticks(xtickdata.values, labels = [x+1 if i in [0,len(xtickdata)-1] else x for (i,x) in enumerate(xtickdata.index)])

### export and finish ###
fname = 'rate_scenarios_apy.png'
fig.savefig(os.path.join(out_dir, fname),bbox_inches='tight')
print(f'Figure saved to {os.path.join(out_dir, fname)}')

plt.show()

In [ ]:
fig = plt.figure(figsize=(24, 6))
ax = fig.add_subplot()
for model in trades_agg.scenario_name.unique():
    model_df = trades_agg.loc[trades_agg.scenario_name==model, :]
    ax = model_df.plot(
        x='day',
        y='trade_volume_usd_sum',
        kind='line',
        ax=ax,
        color=model_colors[model],
        label=model
    )
ax.set_xlabel('Day', fontsize=18)
ax.set_ylabel('Trade volume (USD)', fontsize=18)
ax.tick_params(axis='both', labelsize=12)
ax.grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
ax.ticklabel_format(style='plain', axis='y')
lh = ax.legend(fontsize=18)
fig.savefig(os.path.join(out_dir, 'trade_volume.png'))